1. 데이터 불러오기/정보/결측치 확인
2. 데이터 전처리
3. 훈련/테스트 세트로 데이터 분리
4. 모델 컴파일 및 학습
5. 손실 및 정확도,예측 등 결과 확인 및 분석
6. 성능 튜닝

1-1. 기본적인 라이브러리 불러오기

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

1-2. CSV 파일 불러오고, 파일 정보 및 이상치 탐지

In [ ]:
df = pd.read_csv('abalone.csv')
df

,id,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings
0,0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,15
1,1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7
2,2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9
3,3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10
4,4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7
...,...,...,...,...,...,...,...,...,...,...
4172,4172,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
4173,4173,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
4174,4174,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9
4175,4175,F,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10


In [ ]:
# 데이터 파일 기본 정보 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4177 entries, 0 to 4176
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              4177 non-null   int64  
 1   Sex             4177 non-null   object 
 2   Length          4177 non-null   float64
 3   Diameter        4177 non-null   float64
 4   Height          4177 non-null   float64
 5   Whole_weight    4177 non-null   float64
 6   Shucked_weight  4177 non-null   float64
 7   Viscera_weight  4177 non-null   float64
 8   Shell_weight    4177 non-null   float64
 9   Rings           4177 non-null   int64  
dtypes: float64(7), int64(2), object(1)
memory usage: 326.5+ KB


In [ ]:
df.describe()

,id,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings
count,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000
mean,2088.000000,0.523992,0.407881,0.139516,0.828742,0.359367,0.180594,0.238831,9.933684
std,1205.940366,0.120093,0.099240,0.041827,0.490389,0.221963,0.109614,0.139203,3.224169
min,0.000000,0.075000,0.055000,0.000000,0.002000,0.001000,0.000500,0.001500,1.000000
25%,1044.000000,0.450000,0.350000,0.115000,0.441500,0.186000,0.093500,0.130000,8.000000
50%,2088.000000,0.545000,0.425000,0.140000,0.799500,0.336000,0.171000,0.234000,9.000000
75%,3132.000000,0.615000,0.480000,0.165000,1.153000,0.502000,0.253000,0.329000,11.000000
max,4176.000000,0.815000,0.650000,1.130000,2.825500,1.488000,0.760000,1.005000,29.000000


In [ ]:

# 결측치 확인 값이 False라면 Null 값이 없는거고 True면 있는거니깐
# 해당 행을 제거해주거나 그 열에 해당하는 평균 값을 넣어주기

df.isnull().values.any()

# # 결측치가 있는 행을 제거하려면
# df = df.dropna()

# # 결측치를 열의 평균값으로 채우려면
# df = df.fillna(df.mean())


False

2-1. 문제에 맞는(지도 학습 -분류/회귀, 비지도 학습 - PCA/AutoEncoder) input, target으로 데이터 전처리 하기

In [ ]:
# 전복 csv 데이터 기준으로, 분류 문제는 성별을 맞추는 것, 회귀 문제는 Rings 값을 예측하는 것으로 함

# .values를 넣어 csv 데이터를 넘파이 배열로 바꿔줘야함

# 분류인 경우 target 값은 성별이 됨.
input = df.drop(['id','Sex'],axis=1).values
target = df['Sex']

# 회귀인 경우 target 값은 Rings가 됨.
# input = df.drop(['id','Rings'],axis=1).values
# target = df['Rings']



In [ ]:
# 단, 조건에 맞는 열만 가지고 문제를 수행하라 할 수 도 있음
# 전복 데이터 중 Length, Height, Diameter 만 가지고 수행하라

df = df[['Length','Height','Diameter']]

# 분류 문제에서 target 값이 여러 개가 있을 수 있는데, 그 때 필요한 target 만 쓰라하면
# isin 이라는 함수를 통해 원하는 값의 행 만 추출할 수 있음(전복의 성은 M,F,I 3개 가 있음)
df = df[df['Sex'].isin(['M', 'F'])]


2-2. 입력 데이터 또는 정답 데이터에 수치형 데이터가 아닌 범주형 데이터가 있다면 라벨 인코딩 또는 원-핫 인코딩을 수행해야한다.

In [ ]:
from sklearn.preprocessing import LabelEncoder

# target 을 수치형 데이터로 바꿔주기
le = LabelEncoder()
target = le.fit_transform(target)

# 만약 input에 범주형 데이터가 있다면 필요한 열에 대해서 원-핫 인코딩을 수행해준다.
# df = pd.get_dummies(df,columns=['Sex']).values

In [ ]:
# 만약 오토인코더 문제에서 'activity' 열 전처리: lyingBack, lyingRigh -> 정상 (0), sitting -> 비정상 (1)
# 이러한 유형의 경우는 이렇게 replace를 쓰면 된다,

# df['activity'] = df['activity'].replace({'lyingBack': 0, 'lyingRigh': 0, 'sitting': 1})


2-3. 내가 원하는 데이터를이 최종적으로 잘 추출되었는지 확인

In [ ]:
# unique,shape,value_counts 로 확인 해보기

# df['Sex'].unique(), df.shape, df['Sex'].value_counts()


3-1. 데이터를 train,valid,test 로 나눈다.

In [ ]:
from sklearn.model_selection import train_test_split

# train/test 분리
x_train ,x_test, y_train, y_test = train_test_split (input,target, test_size=0.2,stratify=target)
# train/valid 분리
x_train ,x_valid, y_train, y_valid = train_test_split (x_train,y_train, test_size=0.2,stratify=y_train)



3-2. 데이터의 정규화를 위해 StandardScaler 사용

In [ ]:
from sklearn.preprocessing import StandardScaler

# train 에 대해서한 fit_transform을 적용하며, valid,test는 train에 맞게 transform만 하면 된다.

ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_valid = ss.transform(x_valid)
x_test = ss.transform(x_test)

4-1. 모델 만들기 - 머신러닝 분류

Logistic Regression, Decision Tree, Support Vector Classifier, KNN , Random Forest

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

# 로지스틱 리그레션
lr = LogisticRegression()
lr.fit(x_train,y_train)
lr_pred = lr.predict(x_test)
lr_acc = accuracy_score(y_test,lr_pred)
print(lr_acc)

# 의사결정트리
dt = DecisionTreeClassifier()
dt.fit(x_train,y_train)
dt_pred = dt.predict(x_test)
dt_acc = accuracy_score(y_test,dt_pred)
print(dt_acc)

# 서포트 벡터
svc = SVC()
svc.fit(x_train,y_train)
svc_pred = svc.predict(x_test)
svc_acc = accuracy_score(y_test,svc_pred)
print(svc_acc)

# KNN
knn = KNeighborsClassifier()
knn.fit(x_train,y_train)
knn_pred = knn.predict(x_test)
knn_acc = accuracy_score(y_test,knn_pred)
print(knn_acc)

# 랜덤 포레스트
rf = RandomForestClassifier()
rf.fit(x_train,y_train)
rf_pred = rf.predict(x_test)
rf_acc = accuracy_score(y_test,rf_pred)
print(rf_acc)

4-1. 모델 만들기- 머신러닝 회귀

Linear Regression, Random Forest, Decision Tree,Support Vector Regression, KNN Regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

# 선형 회귀
lr = LinearRegression()
lr.fit(x_train,y_train)
lr_pred = lr.predict(x_test)
lr_mse = mean_squared_error(y_test,lr_pred)
print(lr_mse)

# 의사결정트리
dt = DecisionTreeRegressor()
dt.fit(x_train,y_train)
dt_pred = dt.predict(x_test)
dt_mse = mean_squared_error(y_test,dt_pred)
print(dt_mse)

# 서포트 벡터
svr = SVR()
svr.fit(x_train,y_train)
svr_pred = svr.predict(x_test)
svr_mse = mean_squared_error(y_test,svr_pred)
print(svr_mse)

# KNN
knn = KNeighborsRegressor()
knn.fit(x_train,y_train)
knn_pred = knn.predict(x_test)
knn_mse = mean_squared_error(y_test,knn_pred)
print(knn_mse)

# 랜덤 트리
rf = RandomForestRegressor()
rf.fit(x_train,y_train)
rf_pred = rf.predict(x_test)
rf_mse = mean_squared_error(y_test,rf_pred)
print(rf_mse)

4-1. 딥러닝 모델 만들기(Dense,CNN,LSTM,Transformer)

In [ ]:
# Dense 모델

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

input_dim=x_train.shape[1]

def create_dense_model(input_shape):
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=input_shape))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # 이진 분류일 경우

    # model.add(Dense(10, activation='softmax'))  # 다중 분류일 경우

    # model.add(Dense(1))  # 회귀일 경우

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    # model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  # 다중 분류일 경우(target을 원 핫 인코딩 한 경우)
    # model.compile(optimizer='adam', loss='spares_categorical_crossentropy', metrics=['accuracy'])  # 다중 분류일 경우(target을 라벨 인코딩 한 경우)

    # model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])  # 회귀일 경우
    return model

model = create_dense_model((input_dim,))
model.summary()

In [ ]:
# CNN 모델

# CNN 모델은 기본적으로 3차원 이상의 데이터를 입력으로 받기 때문에
# 필요에 맞게 입력 데이터의 차원을 늘려주어야 한다. 4차원Conv2D를 쓰려면 더 추가!

# train,valid,test에 대해 동일하게 수행
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten

def create_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape))  # 1D 합성곱
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # 이진 분류일 경우

    # model.add(Dense(10, activation='softmax'))  # 다중 분류일 경우

    # model.add(Dense(1))  # 회귀일 경우

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    # model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  # 다중 분류일 경우(target을 원 핫 인코딩 한 경우)
    # model.compile(optimizer='adam', loss='spares_categorical_crossentropy', metrics=['accuracy'])  # 다중 분류일 경우(target을 라벨 인코딩 한 경우)

    # model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])  # 회귀일 경우
    return model

model = create_cnn_model((x_train.shape[1], 1))
model.summary()

In [ ]:
# LSTM 모델
# 시계열 데이터가 필요하므로 데이터를 전처리 해줘야한다.

from collections import Counter

def split_sequences(sequences, n_steps):
    x, y = list(), list()  # 빈 리스트를 생성하여 시퀀스 데이터와 레이블을 담을 공간을 만듦
    for i in range(len(sequences)):  # 전체 시퀀스 데이터를 순회
        # find the end of this pattern
        end_ix = i + n_steps  # 현재 인덱스(i)에서 n_steps만큼 떨어진 시퀀스의 끝을 계산
        # check if we are beyond the dataset
        if end_ix > len(sequences):  # 시퀀스 끝이 데이터의 범위를 넘어서는지 확인
            break  # 범위를 넘으면 루프 종료
        # gather input (X) and output parts (y)
        seq_x = sequences[i:end_ix, :-1]  # 입력 데이터 (특징 데이터)
        seq_y_values = sequences[i:end_ix, -1]  # 시퀀스 동안의 출력 데이터 (레이블들)

        # 가장 빈번하게 나온 레이블 찾기
        most_common_label = Counter(seq_y_values).most_common(1)[0][0]

        x.append(seq_x)  # 입력 데이터 추가
        y.append(most_common_label)  # 가장 많이 나온 레이블 추가

    return np.array(x), np.array(y)  # 리스트를 numpy 배열로 변환하여 반환.

y_train = np.array(y_train)

# 타깃값을 맨 뒤에 붙이기 그래야 함수안에 들어가서 x y 로 나눠줄수가 있음
train_set = np.c_[x_train, y_train]

n_steps = 3
x_train, y_train = split_sequences(train_set, n_steps)

# 이후 분류 문제에 따라 y_train을 인코딩해주기!

# Define the input shape
n_timesteps, n_features, n_outputs = x_train.shape[1], x_train.shape[2], y_train.shape[1]

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(64, activation='relu', input_shape=input_shape))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # 이진 분류일 경우

    # model.add(Dense(10, activation='softmax'))  # 다중 분류일 경우

    # model.add(Dense(1))  # 회귀일 경우

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    # model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  # 다중 분류일 경우(target을 원 핫 인코딩 한 경우)
    # model.compile(optimizer='adam', loss='spares_categorical_crossentropy', metrics=['accuracy'])  # 다중 분류일 경우(target을 라벨 인코딩 한 경우)

    # model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])  # 회귀일 경우
    return model

model = create_lstm_model((n_timesteps, n_features))
model.summary()


In [ ]:
# 트랜스포터 모델, LSTM 모델과 같이 입력 3차원으로 넣어주기

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res

def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_last")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)

input_shape = x_train.shape[1:]
n_classes = len(np.unique(y_train))

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

# 문제에 맞는 손실함수 고를것
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["accuracy"],
)
model.summary()

4-2. 모델 학습

In [ ]:
history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_valid, y_valid))


5-1. 성능 평가

In [ ]:
# 손실 시각화
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

plt.figure(figsize=(10, 6))
plt.plot(epochs, loss, 'b-', label='Training loss')
plt.plot(epochs, val_loss, 'r-', label='Validation loss')
plt.title('Training and Validation Loss')
plt.legend(['train', 'test'], loc='upper right')
plt.xlabel('Epochs')
plt.ylabel('Loss')

In [ ]:
# 정확도 시각화
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(1, len(acc) + 1)

plt.figure(figsize=(10, 6))
plt.plot(epochs, acc, 'b-', label='Training accuracy')
plt.plot(epochs, val_acc, 'r-', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.legend(['train', 'test'], loc='upper right')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')


In [ ]:
# 테스트 셋 평가
model.evaluate(x_test, y_test)


In [ ]:
# confusion matrix,# classification report

from sklearn.metrics import confusion_matrix, classification_report

# 모델 예측
y_pred = model.predict(x_test)
y_pred_labels = np.argmax(y_pred, axis=1)

# 혼동 행렬 계산
cm = confusion_matrix(y_test, y_pred_labels)

# 혼동 행렬 시각화
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel("Predicted labels")
plt.ylabel("True labels")
plt.title("Confusion Matrix")
plt.show()

# 분류 리포트 출력
cr = classification_report(y_test, y_pred_labels)
print(cr)

6. 성능 개량

추가적인 성능을 높이는데는 드롭아웃, 배치 정규화, 밀집 연결층 추가 또는 삭제, 유닛 수 늘리기, 사전 학습 모델로 만들고 층 동결등 미세 조정 등이 있음.